# Projeto 1 - Algoritmos Genéticos

### Base de dados:
(i) http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/

(ii) https://www.math.uwaterloo.ca/tsp/data/index.html

- Considerar as bases Symmetric traveling salesman problem (TSP)

- Desenvolver uma Algoritmo Genético com o framework DEAP

- Utilizar o template Projeto-Algoritmos-Geneticos.ipynb

(i) Investigar os resultados com diferentes operadores:

   - Cruzamento: cxPartialyMatched(), cxUniformPartialyMatched() e cxOrdered()

   - Mutação: mutShuffleIndexes() e mutFlipBit()

   - Seleção: selTournament() e selRoulette()

(ii) Formas de execução:

- Rodar 10 vezes cada combinação distinta de operadores

- Apresentar a melhor solução, média e desvio padrão das 10 rodadas   

- Utilizar pelo menos 5 bases de dados diferentes:

In [ ]:
!pip install deap

In [ ]:
import math
import random
import array
import numpy
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from deap import base
from deap import creator
from deap import algorithms
from deap import tools

In [ ]:
def distL2(x1,y1, x2,y2):
    """Compute the L2-norm (Euclidean) distance between two points.

    The distance is rounded to the closest integer, for compatibility
    with the TSPLIB convention.

    The two points are located on coordinates (x1,y1) and (x2,y2),
    sent as parameters"""
    xdiff = x2 - x1
    ydiff = y2 - y1
    return int(math.sqrt(xdiff*xdiff + ydiff*ydiff) + .5)

In [ ]:
def distL1(x1,y1, x2,y2):
    """Compute the L1-norm (Manhattan) distance between two points.

    The distance is rounded to the closest integer, for compatibility
    with the TSPLIB convention.

    The two points are located on coordinates (x1,y1) and (x2,y2),
    sent as parameters"""
    return int(abs(x2-x1) + abs(y2-y1)+.5)

In [ ]:
def calc_matriz(coord, dist):
    """Calcula a matriz de distância entre as cidades.

    Utiliza a função armazena em 'dist' para calcular as distâncias
    entre dois pontos quaisquer.

    Parametross:
    -coord -- lista de tuplas com as coordenadas de todos os pontos, [(x1,y1),...,(xn,yn)]
    -dist -- funçaõ distância
    """
    n = len(coord)
    # dicionario com as com coordenadas (chave) e distâncias (valor)
    D = {}
    for i in range(n-1):
        for j in range(i+1,n):
            (x1,y1) = coord[i]
            (x2,y2) = coord[j]
            D[i,j] = dist(x1,y1,x2,y2)
            D[j,i] = D[i,j]
    return n,D

In [ ]:
def read_tsplib(filename):
    "basic function for reading a TSP problem on the TSPLIB format"
    "NOTE: only works for 2D euclidean or manhattan distances"
    f = open(filename, 'r');

    line = f.readline()
    while line.find("EDGE_WEIGHT_TYPE") == -1:
        line = f.readline()

    if line.find("EUC_2D") != -1:
        dist = distL2
    elif line.find("MAN_2D") != -1:
        dist = distL1
    else:
        print("cannot deal with non-euclidean or non-manhattan distances")
        raise Exception

    while line.find("NODE_COORD_SECTION") == -1:
        line = f.readline()

    xy_positions = []
    while 1:
        line = f.readline()
        if line.find("EOF") != -1: break
        (i,x,y) = line.split()
        # print(dist,i,x,y)
        x = float(x)
        y = float(y)
        xy_positions.append((x,y))

    n,D = calc_matriz(xy_positions, dist)
    return n, xy_positions, D

In [ ]:
def custo(tour, D):
    """Calcula a distÇancia da rota de acordo com a matriz 'D'."""
    z = D[tour[-1], tour[0]]    # aresta a entre a primeira e última cidade da rota
    for i in range(1,len(tour)):
        z += D[tour[i], tour[i-1]] # atualiza o custo da rota a partir da cidade i-1 to i
    return z

In [ ]:
def randtour(n):
    """Gera uma solução aleatória de tamanho 'n'."""
    sol = list(range(n)) # Gera uma lista sequencial
    random.shuffle(sol) # Embaralha a lista
    return sol


In [ ]:
# Exemplo com instância "a280.tsp"
dataset = 'eil51'
path = './'+ dataset + '.tsp'
print(path)

best_fitness_dataset = {
    'eil51' : '426',
    'st70' : '675',
    'rat99' : '1211',
    'rd100': '7910',
    'pr76': '108159',
}

n, coord, D = read_tsplib(path)
n, coord, D

In [ ]:
""" Registro dos operadores de combinação
    -toolbox -- caixa de ferramentas do DEAP
    -v_indpb -- probabilidade de mutação
"""
def cx_partialy_matched(toolbox, v_indpb = 0.01):
    toolbox.register("mate", tools.cxUniformPartialyMatched, indpb = v_indpb)

""" Registro dos operadores de combinação
    -toolbox -- caixa de ferramentas do DEAP
"""
def cx_uniform_partialy_matched(toolbox):
    toolbox.register("mate", tools.cxPartialyMatched)

""" Registro dos operadores de combinação
    -toolbox -- caixa de ferramentas do DEAP
"""
def cx_ordered(toolbox):
    toolbox.register("mate", tools.cxOrdered)

In [ ]:
""" Registro dos operadores de mutação
    -toolbox -- caixa de ferramentas do DEAP
    -v_indpb -- probabilidade de mutação
"""
def mut_shuffle_indexes(toolbox, v_indpb = 0.01 ):
    toolbox.register("mutate", tools.mutShuffleIndexes, indpb = v_indpb)

In [ ]:
""" Registro dos operadores de seleção
    -toolbox -- caixa de ferramentas do DEAP
    -v_tournsize -- tamanho do torneio
"""
def sel_tournament(toolbox, v_tournsize = 4):
    toolbox.register("select", tools.selTournament, tournsize = v_tournsize)

""" Registro dos operadores de seleção
    -toolbox -- caixa de ferramentas do DEAP
"""
def sel_toulette(toolbox):
    toolbox.register("select", tools.selRoulette)


In [ ]:
def eval(individual):
    """Função de avaliação.
    """
    return (custo(individual, D),)

In [ ]:
def create_individual(n):
    """Função que cria um indivíduo.
    """
    return creator.Individual(randtour(n))

In [ ]:
def create_fitness():
    """Função que cria o tipo de função fitness.
    """
    # Cria o tipo de função fitness
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,)) # minimizar = peso negativo
    creator.create("Individual", list, fitness=creator.FitnessMin)

In [ ]:
# Cria a toolbox
def create_toolbox():
    """Função que cria a toolbox do DEAP.
    """
    toolbox = base.Toolbox()

    # Registra o gene e a função de criação
    toolbox.register("gene", create_individual, n)

    # Registra a populacao na toolbox
    toolbox.register("population", tools.initRepeat, list, toolbox.gene)

    # Registra o evaluate na toolbox
    toolbox.register("evaluate", eval)

    return toolbox

In [ ]:
def test_register(toolbox):
    # Testando os registros
    pop = []
    pop.append(toolbox.gene())

    # pop = toolbox.population(n=10)
    fitValue = list(map(toolbox.evaluate,pop))
    print(f'{pop[0]}: {fitValue}')

In [ ]:
def statistics(individual):
    return individual.fitness.values

In [ ]:

def run_GA(toolbox,
           prob_cross,
           prob_mut,
           halloffame,
           generations,
           population_n = 400,
           v_verbose = True):
    """Executa o algoritmo genético.

    Parâmetros:
    -toolbox -- caixa de ferramentas do DEAP
    -prob_cross -- probabilidade de cruzamento
    -prob_mut -- probabilidade de mutação
    -halloffame -- objeto do DEAP que armazena o melhor indivíduo encontrado
    -geracoes -- número de gerações
    -population_n -- tamanho da população
    -verbose -- se True, imprime informações sobre a execução do algoritmo

    """
    population = toolbox.population(n = population_n)

    stats = tools.Statistics(statistics)

    stats.register("min", numpy.min)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)

    population, info = algorithms.eaSimple(population,
                                               toolbox,
                                               prob_cross,
                                               prob_mut,
                                               generations,
                                               stats,
                                               halloffame,
                                               verbose = v_verbose)

    return min(info, key=lambda x: (x['min']))


In [ ]:
def plot_TSP(individual, names ,n):
    """Visualizar o melhor caminho encontrado pelo algoritmo genético.

    Parâmetros:
    -individual -- melhor indivíduo encontrado pelo algoritmo genético
    -names -- nomes dos operadores genéticos utilizados
    -n -- número da figura
    """
    plt.figure(n)
    plt.title(names[n])
    plt.scatter(*zip(*coord), marker='.', color='red')
    locs = [coord[i] for i in individual]
    locs.append(locs[0])
    plt.plot(*zip(*locs), linestyle='-', color='blue')
    plt.text(-4, -12, f'caminho: {individual}, custo: {custo(individual, D)}', fontsize=12, wrap=True, verticalalignment='top')


In [ ]:
def run_operators(prob_cross, prob_mut, generations, crossover, mutation, selection):
    """Executa o algoritmo genético para uma série de execuções (10 no caso), usando diferentes combinações de operadores genéticos (cruzamento, mutação e seleção)

    Paramentros:
    -prob_cross -- probabilidade de cruzamento
    -prob_mut -- probabilidade de mutação
    -generations -- número de gerações
    -crossover -- função de cruzamento
    -mutation -- função de mutação
    -selection -- função de seleção
    """

    best_fit = []

    halloffame = tools.HallOfFame(1) # armazena o melhor indivíduo encontrado durante a execução do algoritmo genético

    for _ in range(10):
        toolbox = create_toolbox()
        selection(toolbox)
        crossover(toolbox)
        mutation(toolbox)
        #test_register(toolbox)
        best = run_GA(toolbox, prob_cross, prob_mut, halloffame, generations) #executa o algoritmo genético usando a toolbox configurada e os parâmetros fornecidos
        best_fit.append(best) #Adiciona o melhor indivíduo encontrado à lista best_fit

    return min(best_fit, key=lambda x: (x['min'])), halloffame[0]

In [ ]:
#Parametros de execução dos Operadores Genéticos e Avaliação do Desempenho
prob_cross = 0.95
prob_mut = 0.3
generations = 500

create_fitness()

crossover_functions = [cx_ordered,cx_partialy_matched, cx_uniform_partialy_matched]
mutation_functions = [mut_shuffle_indexes]
selection_functions = [sel_tournament, sel_toulette]

best_record = []
best_path = []
collumn_names = []

#Loop para Testar Diferentes Combinações de Operadores Genéticos
for crossover in crossover_functions:
    for mutation in mutation_functions:
        for selection in selection_functions:
            collumn_names.append(f'{crossover.__name__}/{mutation.__name__}/{selection.__name__}')
            record, path = run_operators(prob_cross, prob_mut, generations, crossover, mutation, selection)
            best_record.append(record)
            best_path.append(path)


### Resultados de melhor solução, média e desvio padrão de cada combinação de operadores

In [ ]:
print("[TSPLIB] Melhor solução global do dataset: " + best_fitness_dataset[dataset])

In [ ]:
#Tabela da execução do algoritmo genético para diferentes combinações de operadores genéticos, armazenando as métricas de desempenho em um DataFrame.
dt = pd.DataFrame(best_record).T
dt.columns = collumn_names
print(dt.loc[['min','avg','std']])

### Caminhos das melhores soluções de cada combinação de operadores

In [ ]:
#visualizações das rotas encontradas pelo algoritmo genético para diferentes combinações de operadores genéticos
for i in range(len(best_path)):
    plot_TSP(best_path[i], collumn_names, i)